In [2]:
import pandas as pd
import numpy as np
import subprocess
import tempfile
import os

In [3]:
train_filepath = "train_workspace/"
animalList = ["Human","Pig","Chicken","Rat","Mouse","Dog"]
levelList = ["Strict","Relaxed","Intermediate"]
dataList = ["Ohnologs","No-Ohnologs","Paralogs"]
dataNameList = ["ohnologs","no-ohnologs","paralog"]

In [4]:
working_level = "Intermediate"

In [5]:
def get_df_dict(level):
    df_dict = {}    
    current_file_path = train_filepath + level + "/"
    for (dataName,data) in zip(dataNameList,dataList):                
        df_dict[data] = pd.read_pickle(current_file_path + dataName + "-complete_v2.pkl")        
    return df_dict

In [6]:
df_dict = get_df_dict(working_level)
df_dict["Ohnologs"].head()

,Is_Ohnolog,Is_Paralog,Sequence-1,Sequence-1 GC,Sequence-1 Id,Sequence-1 Length,Sequence-1-Transcript Id,Sequence-2,Sequence-2 GC,Sequence-2 Id,Sequence-2 Length,Sequence-2-Transcript Id,Species,Sequence-1-Transcript-Version,Sequence-2-Transcript-Version,Sequence-1-Transcript-Biotype,Sequence-2-Transcript-Biotype,Sequence-1-Chromosome,Sequence-2-Chromosome
0,1,0.0,CTTTGGAAGTCCTATGAGGGACCATTTACGGTTTCCTCAGTAATTT...,0.429090,ENSG00000095464,3307,ENST00000371447,AGTATGTTTTGCAGACAAGACCCAGAGAAGTCCAGACTGGACTTGT...,0.469856,ENSG00000132915,5706,ENST00000508173,Human,3,5,protein_coding,retained_intron,10,5
1,1,0.0,CGTTTTGGCAAGGGATTAAAGTGCTCCCCCCTGTGGCAGCAGTGAC...,0.441268,ENSG00000077684,5772,ENST00000226319,ATACAATAGTGCTCCGCGCCGCCTCAGCCGCCGCCGCCGCCCAACC...,0.450953,ENSG00000102221,4934,ENST00000614628,Human,10,4,protein_coding,protein_coding,4,X
2,1,0.0,AGTCAACCTCTGGAAGTAAGTCAACTCCATTCTGAAAAAGAAGAGT...,0.357972,ENSG00000109158,11973,ENST00000264318,ACATAATCTAAGACCACAAACCACCTTGTTCCACGTGAGAAGGAAA...,0.402006,ENSG00000145863,2393,ENST00000523217,Human,3,5,protein_coding,protein_coding,4,5
3,1,0.0,ACCACAGAGGCGTCTGGCTAACTCATCTCCAGACCTAAGTTGGGAA...,0.572634,ENSG00000137216,3318,ENST00000259746,CAGTCTCCTGCCATGCAAAGAAGGCTCATAGAGTTGCTTTGGAAGT...,0.516923,ENSG00000196187,5407,ENST00000474478,Human,13,5,protein_coding,processed_transcript,6,1
4,1,0.0,GGGCCTCCTCCCTGGGGTGTGAGCAGGTCGGCGCGCCACACTTCTC...,0.468779,ENSG00000113396,3219,ENST00000262462,AGTCCTGCCCGGAACCCCCGGCAACGCGCATACGACTACACCTGCT...,0.489140,ENSG00000140284,2394,ENST00000267842,Human,8,9,protein_coding,protein_coding,5,15


In [7]:
def generate_fasta_file(gene_id,transcript_id,sequence,version,biotype,file):
    header = ">" + transcript_id + "." + str(version) + " "+ gene_id + ":" + transcript_id + "." + str(version) + " cdna:" + biotype + "\n"
    file.write(header + sequence)    

In [10]:
columns_name = ["Querry_Seq_Id","Subject_Seq_Id","Percent Identical Matches","Aligment Length","Nr Mismatch","Nr Gap Open","Start Align",
               "End Align","Start Align Sub","End Align Sub","Evalue","Bit Score"]
relevant_columns = ["Percent Identical Matches","Aligment Length","Nr Mismatch","Nr Gap Open","Evalue","Bit Score"]

blast_types = ["High","Medium","Low","Total"]
relevant_col_Total = [i + "_Total" for i in relevant_columns] 
relevant_col_High = [i + "_High" for i in relevant_columns]
relevant_col_Medium = [i + "_Medium" for i in relevant_columns]
relevant_col_Low = [i + "_Low" for i in relevant_columns]
total_columns = (relevant_col_High + relevant_col_Medium + relevant_col_Low + relevant_col_Total)

df_blast = {}
for dataName in dataList:
    df = df_dict[dataName]    
    df_blast[dataName] = pd.DataFrame(columns=total_columns)
    
    list_dict_cols = {}
    list_count = []
    for colName in total_columns:
        list_dict_cols[colName] = []
        
    for index, row in df.iterrows():     
        with open("gene1.fa","w+") as f1,open("gene2.fa","w+") as f2:      
            f1.seek(0)
            f1.truncate()
            f2.seek(0)
            f2.truncate()
            generate_fasta_file(row["Sequence-1 Id"],row["Sequence-1-Transcript Id"],row["Sequence-1"],row["Sequence-1-Transcript-Version"],row["Sequence-1-Transcript-Biotype"],f1)
            generate_fasta_file(row["Sequence-2 Id"],row["Sequence-2-Transcript Id"],row["Sequence-2"],row["Sequence-2-Transcript-Version"],row["Sequence-2-Transcript-Biotype"],f2)                 
        list_args = ['anaconda3/envs/thesis3/bin/blastn','-query','gene1.fa','-subject','gene2.fa','-word_size','11','-outfmt','10']
        blast_querry = subprocess.Popen(list_args, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT,
        universal_newlines=True)
        stdout,stderr = blast_querry.communicate()      
        df_temp = pd.DataFrame([i.split(',') for i in stdout.split('\n') if i != ''],columns=columns_name)
        df_temp = df_temp[relevant_columns]
        
        df_temp[relevant_columns] = df_temp[relevant_columns].astype(float)
                
        matches_high = df_temp.loc[(df_temp["Evalue"]<=1e-50)]
        matches_medium = df_temp.loc[(df_temp["Evalue"]>1e-50) & (df_temp["Evalue"]<0.01)]
        matches_low = df_temp.loc[(df_temp["Evalue"]>0.01) & (df_temp["Evalue"]<10)]
              
        df_dict_level = {}
        df_dict_level["High"] = matches_high
        df_dict_level["Medium"] = matches_medium
        df_dict_level["Low"] = matches_low
        df_dict_level["Total"] = df_temp
        
        list_count.append(len(df_temp))
        
        for blast_type in blast_types:
            for relevant_column in relevant_columns:
                list_dict_cols[relevant_column + "_" + blast_type].append(df_dict_level[blast_type][relevant_column].mean())
                
    df_blast[dataName]=pd.DataFrame.from_dict(list_dict_cols,orient='index').transpose()
    df_blast[dataName]["Nr Hits"] = list_count                    
        

In [11]:
df_ohnologs_no_nan  = df_blast["Ohnologs"].fillna(0)
df_ohnologs_no_nan.head()

,Percent Identical Matches_High,Aligment Length_High,Nr Mismatch_High,Nr Gap Open_High,Evalue_High,Bit Score_High,Percent Identical Matches_Medium,Aligment Length_Medium,Nr Mismatch_Medium,Nr Gap Open_Medium,...,Nr Gap Open_Low,Evalue_Low,Bit Score_Low,Percent Identical Matches_Total,Aligment Length_Total,Nr Mismatch_Total,Nr Gap Open_Total,Evalue_Total,Bit Score_Total,Nr Hits
0,73.058,823.5,194.0,23.0,1.425000e-53,272.0,0.0000,0.0,0.00,0.00,...,0.000000,4.828000,22.273333,96.042529,107.352941,22.941176,2.705882,4.260000,51.652941,17
1,78.307,567.0,117.0,6.0,5.180000e-102,361.0,85.7505,31.5,4.50,0.00,...,0.272727,2.195455,23.963636,95.077926,35.592593,4.925926,0.518519,5.709458,35.981481,27
2,77.410,726.0,154.0,9.0,6.540000e-121,424.0,76.9230,104.0,24.00,0.00,...,0.166667,2.126667,23.900000,92.587125,114.250000,22.500000,1.250000,1.595000,78.450000,8
3,0.000,0.0,0.0,0.0,0.000000e+00,0.0,86.0485,45.5,6.50,0.00,...,0.000000,4.424444,22.344444,97.938150,15.200000,0.750000,0.000000,3.982011,25.025000,20
4,0.000,0.0,0.0,0.0,0.000000e+00,0.0,79.3405,90.0,19.25,1.25,...,0.000000,1.409714,23.271429,90.491909,42.818182,7.818182,0.454545,0.897455,33.854545,11


In [12]:
df_no_ohnologs_no_nan = df_blast["No-Ohnologs"].fillna(0)
df_no_ohnologs_no_nan.head()

,Percent Identical Matches_High,Aligment Length_High,Nr Mismatch_High,Nr Gap Open_High,Evalue_High,Bit Score_High,Percent Identical Matches_Medium,Aligment Length_Medium,Nr Mismatch_Medium,Nr Gap Open_Medium,...,Nr Gap Open_Low,Evalue_Low,Bit Score_Low,Percent Identical Matches_Total,Aligment Length_Total,Nr Mismatch_Total,Nr Gap Open_Total,Evalue_Total,Bit Score_Total,Nr Hits
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,2.662857,22.728571,98.979571,12.142857,0.142857,0.000000,2.662857,22.728571,7
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.988250,22.918750,98.314375,12.750000,0.312500,0.000000,1.988250,22.918750,16
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,3.200000,21.400000,100.000000,11.000000,0.000000,0.000000,3.200000,21.400000,6
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.416667,2.104833,23.575000,97.439083,14.000000,0.083333,0.416667,2.104833,23.575000,12


In [13]:
df_paralogs_no_nan = df_blast["Paralogs"].fillna(0)
df_paralogs_no_nan.head()

,Percent Identical Matches_High,Aligment Length_High,Nr Mismatch_High,Nr Gap Open_High,Evalue_High,Bit Score_High,Percent Identical Matches_Medium,Aligment Length_Medium,Nr Mismatch_Medium,Nr Gap Open_Medium,...,Nr Gap Open_Low,Evalue_Low,Bit Score_Low,Percent Identical Matches_Total,Aligment Length_Total,Nr Mismatch_Total,Nr Gap Open_Total,Evalue_Total,Bit Score_Total,Nr Hits
0,0.0,0.0,0.0,0.0,0.0,0.0,82.157,48.0,7.5,1.0,...,0.250000,0.700000,22.350000,91.089333,25.333333,2.833333,0.500000,0.466690,28.816667,6
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.272727,5.463636,23.627273,96.667091,14.363636,0.272727,0.272727,5.463636,23.627273,11
2,0.0,0.0,0.0,0.0,0.0,0.0,76.471,102.0,22.0,2.0,...,0.200000,1.722300,22.900000,96.075273,21.000000,2.090909,0.363636,1.565727,25.790909,11
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,...,0.272727,3.754545,22.254545,97.296182,13.090909,0.181818,0.272727,3.754545,22.254545,11
4,0.0,0.0,0.0,0.0,0.0,0.0,88.889,27.0,3.0,0.0,...,0.100000,5.085250,22.615000,96.371476,16.619048,1.428571,0.095238,4.843143,23.176190,21


In [14]:
df_paralogs_no_nan.tail()

,Percent Identical Matches_High,Aligment Length_High,Nr Mismatch_High,Nr Gap Open_High,Evalue_High,Bit Score_High,Percent Identical Matches_Medium,Aligment Length_Medium,Nr Mismatch_Medium,Nr Gap Open_Medium,...,Nr Gap Open_Low,Evalue_Low,Bit Score_Low,Percent Identical Matches_Total,Aligment Length_Total,Nr Mismatch_Total,Nr Gap Open_Total,Evalue_Total,Bit Score_Total,Nr Hits
22966,83.019,212.0,34.0,2.0,9.440000e-51,191.0,85.039667,53.333333,9.0,1.222222,...,0.128571,2.190800,24.791429,94.930687,21.750000,1.925000,0.275000,1.917138,28.523750,80
22967,78.030,396.0,85.0,2.0,8.890000e-69,248.0,73.984000,369.000000,83.0,11.000000,...,0.000000,0.590000,23.300000,89.780000,136.500000,28.333333,2.166667,0.393333,79.700000,6
22968,0.000,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.079000,21.400000,100.000000,11.000000,0.000000,0.000000,0.079000,21.400000,1
22969,0.000,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000,...,0.260870,3.646522,22.869565,97.518304,13.826087,0.260870,0.260870,3.646522,22.869565,23
22970,0.000,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000,...,0.200000,4.832000,22.520000,96.644800,13.600000,0.400000,0.200000,4.832000,22.520000,5


In [15]:
df_dict["Paralogs"].head()

,Is_Ohnolog,Is_Paralog,Sequence-1,Sequence-1 GC,Sequence-1 Id,Sequence-1 Length,Sequence-1-Transcript Id,Sequence-2,Sequence-2 GC,Sequence-2 Id,Sequence-2 Length,Sequence-2-Transcript Id,Species,Sequence-1-Transcript-Version,Sequence-2-Transcript-Version,Sequence-1-Transcript-Biotype,Sequence-2-Transcript-Biotype,Sequence-1-Chromosome,Sequence-2-Chromosome
499,0,1.0,GTCGTCCGCAAAGCCTGAGTCCTGTCCTTTCTCTCTCCCCGGACAG...,0.556992,ENSG00000111057,2088,ENST00000549078,ATGACCTCCTTCTACAGCACCTCCTCATGCCCTCTGGGTTGCACCA...,0.588475,ENSG00000108417,1475,ENST00000225550,Human,5,3,retained_intron,protein_coding,12,17
1029,0,1.0,AGAGCGGGCGGGGCGGACACGCTTAGCGTAAGCGCAGGGGCCAGGA...,0.418398,ENSG00000164794,6979,ENST00000524391,GGAGCCGGCGCGGCCGCGGGCCAGGCGCCGAGGTGCGCGCGGAGCG...,0.643981,ENSG00000075043,9213,ENST00000626839,Human,5,2,protein_coding,protein_coding,8,20
3517,0,1.0,AGCGGGCGGCGAGCAGCGCGGAGCGCGCGGGGCACCGAGGAGCGGG...,0.622584,ENSG00000100314,3259,ENST00000216144,CATAAGCAGCTTTATGGGTCCTGAAAGCCAAGGGTGCCCAGGGGTG...,0.548652,ENSG00000175544,2559,ENST00000325656,Human,3,6,protein_coding,protein_coding,22,11
4305,0,1.0,CACATGCTGGACGGATTCTGGGTCTGCTCTTTGTGCAAACAAAACA...,0.432248,ENSG00000184588,4546,ENST00000423207,TGCGTGCCTGGAGCAGCAGCGTCTCCAGGGACAGGCAGCCACCATG...,0.555183,ENSG00000133256,3280,ENST00000255622,Human,6,10,protein_coding,protein_coding,1,4
4123,0,1.0,CTGTTCCAGACTTCTCAGGGTTTTTACAGCACAAACAACTGCATAC...,0.432591,ENSG00000151572,3909,ENST00000392979,AGGAGCACCCGCCGCTGGCAGCCACTCTCCTGGCCGCCAAACCCCA...,0.641152,ENSG00000185101,5487,ENST00000532094,Human,7,5,protein_coding,retained_intron,12,11


In [16]:
name_data_dict = {}
name_data_dict["Ohnologs"] = "ohnologs"
name_data_dict["No-Ohnologs"] = "no-ohnologs"
name_data_dict["Paralogs"] = "paralog"

df_data_dict = {}
df_data_dict["Ohnologs"] = df_ohnologs_no_nan
df_data_dict["No-Ohnologs"] = df_paralogs_no_nan
df_data_dict["Paralogs"] = df_paralogs_no_nan

for dataName in dataList:
    df_save = pd.concat([df_dict[dataName].reset_index(drop=True),df_data_dict[dataName].reset_index(drop=True)],axis=1,sort=False)
    df_save.to_pickle(train_filepath + working_level + "/" + name_data_dict[dataName] + "-complete2.pkl")
    

df_save.head()

,Is_Ohnolog,Is_Paralog,Sequence-1,Sequence-1 GC,Sequence-1 Id,Sequence-1 Length,Sequence-1-Transcript Id,Sequence-2,Sequence-2 GC,Sequence-2 Id,...,Nr Gap Open_Low,Evalue_Low,Bit Score_Low,Percent Identical Matches_Total,Aligment Length_Total,Nr Mismatch_Total,Nr Gap Open_Total,Evalue_Total,Bit Score_Total,Nr Hits
0,0,1.0,GTCGTCCGCAAAGCCTGAGTCCTGTCCTTTCTCTCTCCCCGGACAG...,0.556992,ENSG00000111057,2088,ENST00000549078,ATGACCTCCTTCTACAGCACCTCCTCATGCCCTCTGGGTTGCACCA...,0.588475,ENSG00000108417,...,0.250000,0.700000,22.350000,91.089333,25.333333,2.833333,0.500000,0.466690,28.816667,6
1,0,1.0,AGAGCGGGCGGGGCGGACACGCTTAGCGTAAGCGCAGGGGCCAGGA...,0.418398,ENSG00000164794,6979,ENST00000524391,GGAGCCGGCGCGGCCGCGGGCCAGGCGCCGAGGTGCGCGCGGAGCG...,0.643981,ENSG00000075043,...,0.272727,5.463636,23.627273,96.667091,14.363636,0.272727,0.272727,5.463636,23.627273,11
2,0,1.0,AGCGGGCGGCGAGCAGCGCGGAGCGCGCGGGGCACCGAGGAGCGGG...,0.622584,ENSG00000100314,3259,ENST00000216144,CATAAGCAGCTTTATGGGTCCTGAAAGCCAAGGGTGCCCAGGGGTG...,0.548652,ENSG00000175544,...,0.200000,1.722300,22.900000,96.075273,21.000000,2.090909,0.363636,1.565727,25.790909,11
3,0,1.0,CACATGCTGGACGGATTCTGGGTCTGCTCTTTGTGCAAACAAAACA...,0.432248,ENSG00000184588,4546,ENST00000423207,TGCGTGCCTGGAGCAGCAGCGTCTCCAGGGACAGGCAGCCACCATG...,0.555183,ENSG00000133256,...,0.272727,3.754545,22.254545,97.296182,13.090909,0.181818,0.272727,3.754545,22.254545,11
4,0,1.0,CTGTTCCAGACTTCTCAGGGTTTTTACAGCACAAACAACTGCATAC...,0.432591,ENSG00000151572,3909,ENST00000392979,AGGAGCACCCGCCGCTGGCAGCCACTCTCCTGGCCGCCAAACCCCA...,0.641152,ENSG00000185101,...,0.100000,5.085250,22.615000,96.371476,16.619048,1.428571,0.095238,4.843143,23.176190,21
